In [2]:
import numpy as np
import pandas as pd
import time as time
import operator
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, Image, clear_output
from scipy.spatial.distance import pdist, squareform
from collections import defaultdict

In [44]:
filepath = '/Users/alfredogarbuno/github-repos/hashcode-2016/qualif/data/busy_day.in'
f = open(filepath, "r+")

rows, cols, ndrones, deadline, maxload = f.readline().split()
ntypes = int(f.readline())
weights = np.array(f.readline().split(), np.int32)
nwhs = int(f.readline())

stocks = np.empty([nwhs, ntypes] , np.int32)
whloc  = np.empty([nwhs, 2], np.float32)

for wh in range(nwhs):
    whloc[wh]  = np.array(f.readline().split(), np.int32)
    stocks[wh] = np.array(f.readline().split(), np.int32)

norders = int(f.readline())
orders = defaultdict(list)
destinations = np.empty([norders, 2], np.int32)
nitemsorders = defaultdict(int)

for order in range(norders):
    destinations[order] = np.array(f.readline().split(), np.int32)
    nitemsorders[order] = int(f.readline())
    orders[order] = np.int_(f.readline().split())

f.close()